In [1]:
# Import necessary modules
import os

import pandas as pd
from dotenv import load_dotenv
from modules.paragraph_generation import generate_paragraph, generate_advertising_paragraph
from modules.data_processing import load_and_filter_data, preprocess_columns_simple, enrich_advertiser_data_with_domain
from modules.linkedin_requests import initialize_client, generate_linkedin_request
# Load API key
load_dotenv()
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')

In [2]:
# Load and preprocess datasets
people_path = './data/YC F24 Founder People.csv'
personal_data_path = './data/Sudarshan Sridharan People.csv'

relevant_columns = [
    'full_name', 'title', 'location', 'company', 'company_linkedin',
    'company_description', 'sub_title', 'summary', 'current_job_description',
    'person_industry', 'skills', 'education', 'work_history', 'company_domain'
]

people_data = load_and_filter_data(people_path, relevant_columns)
people_data = preprocess_columns_simple(people_data)

personal_data = load_and_filter_data(personal_data_path, relevant_columns)
personal_data = preprocess_columns_simple(personal_data)
# Enrich personal data with domain information
personal_data = enrich_advertiser_data_with_domain(personal_data)
# Generate paragraphs
people_data['descriptive_paragraph'] = people_data.apply(generate_paragraph, axis=1)
personal_data['advertising_paragraph'] = personal_data.apply(generate_advertising_paragraph, axis=1)

In [3]:
personal_data

,full_name,title,location,company,company_linkedin,company_description,sub_title,summary,current_job_description,person_industry,skills,education,work_history,company_domain,domain_info,advertising_paragraph
0,Sudarshan Sridharan,Founder,"San Francisco Bay, San Francisco Bay Area, Uni...",Pipeline,https://www.linkedin.com/company/withpipeline,"Get more leads, close more deals. Pipeline set...","CEO, Pipeline",NaN,Pipeline puts LinkedIn prospecting and outreac...,"Technology, Information and Internet",Investing,Clemson University,"Pipeline, Fion Technologies, SaveMAPS, Second ...",withpipeline.com,Sign Up → Sign Up → NEW Supercharge Your Sales...,Sudarshan Sridharan is the Founder at Pipeline...


In [4]:
# Load the system prompt from a .txt file
with open('system_prompt.txt', 'r') as file:
    system_prompt = file.read()

# Initialize Anthropic client
client = initialize_client(ANTHROPIC_API_KEY)

# Use the first advertiser's data for testing
advertising_paragraph = personal_data['advertising_paragraph'].iloc[0]
domain_info = personal_data['domain_info'].iloc[0]

# Process only the first 10 entries
max_entries = 10

print("Generated LinkedIn Requests:\n" + "=" * 30)

for idx, row in people_data.iterrows():
    if idx >= max_entries:
        break
    
    descriptive_paragraph = row['descriptive_paragraph']
    linkedin_request = generate_linkedin_request(client, advertising_paragraph, descriptive_paragraph, domain_info, system_prompt)
    
    # Format the output nicely
    from_name = "Advertiser Name"  # Replace with the actual advertiser's name if available
    to_name = row['full_name']  # Assuming 'full_name' column exists for the person being contacted
    
    print(f"From: {from_name}")
    print(f"To: {to_name}")
    print(f"Message: {linkedin_request}\n")
    print("-" * 30)


Generated LinkedIn Requests:
From: Advertiser Name
To: Edward Aryee
Message: Hi Edward! Fellow startup founder here. Noticed your impressive journey from Google to founding SRE.ai. I'm building Pipeline to help tech founders with outreach automation. Would love to connect and exchange insights about scaling B2B solutions in the YC ecosystem.

------------------------------
From: Advertiser Name
To: Victor Cheng
Message: Hey Victor! Fellow founder here - noticed you're building vly.ai through YC F24. As someone who's also in the B2B tech space, I'd love to connect and share insights about scaling startups. Your tech stack expertise is impressive!

------------------------------
From: Advertiser Name
To: Daryl Budiman
Message: Hi Daryl! Fellow startup founder here - noticed you're building Andoria through YC. As someone developing automation tools for LinkedIn outreach at Pipeline, I'd love to connect and explore potential synergies between our ventures. Your technical background really 